In [33]:
import pandas as pd
import numpy as np
import math
import warnings
warnings.filterwarnings('ignore')

In [41]:
dataset = pd.read_csv("Churn_Modelling.csv")
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [42]:
data=pd.get_dummies(dataset)
data.head()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,...,Surname_Zubarev,Surname_Zubareva,Surname_Zuev,Surname_Zuyev,Surname_Zuyeva,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,1,15634602,619,42,2,0.00,1,1,1,101348.88,...,0,0,0,0,0,1,0,0,1,0
1,2,15647311,608,41,1,83807.86,1,0,1,112542.58,...,0,0,0,0,0,0,0,1,1,0
2,3,15619304,502,42,8,159660.80,3,1,0,113931.57,...,0,0,0,0,0,1,0,0,1,0
3,4,15701354,699,39,1,0.00,2,0,0,93826.63,...,0,0,0,0,0,1,0,0,1,0
4,5,15737888,850,43,2,125510.82,1,1,1,79084.10,...,0,0,0,0,0,0,0,1,1,0


In [43]:
def feature_scaling(data):
    for i in data.columns:
        data[i]=(data[i]-data[i].min())/(data[i].max()-data[i].min())
    return data
data=feature_scaling(data)
data.head()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,...,Surname_Zubarev,Surname_Zubareva,Surname_Zuev,Surname_Zuyev,Surname_Zuyeva,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,0.0000,0.275616,0.538,0.324324,0.2,0.000000,0.000000,1.0,1.0,0.506735,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.0001,0.326454,0.516,0.310811,0.1,0.334031,0.000000,0.0,1.0,0.562709,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,0.0002,0.214421,0.304,0.324324,0.8,0.636357,0.666667,1.0,0.0,0.569654,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.0003,0.542636,0.698,0.283784,0.1,0.000000,0.333333,0.0,0.0,0.469120,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.0004,0.688778,1.000,0.337838,0.2,0.500246,0.000000,1.0,1.0,0.395400,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [44]:
data.shape

(10000, 2948)

In [37]:
#Handling categorical data
categorical_data = []
for name in list(data.columns):
    if is_numeric_dtype(data[name]):
        pass
    else:
        categorical_data.append(name)


data = pd.get_dummies(data, columns=categorical_data)
data.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,1,0


In [38]:
data.shape

(10000, 14)

In [46]:
def split_train_test(data):
    train=len(data)*0.70
    train=int(train)
    test=len(data)-train
    test=int(test)
    
    train=data.head(train)
    test=data.tail(test)
    
    return train,test
train,test=split_train_test(data)

In [49]:
def separate(data):
    output=data.Exited
    return data.drop('Exited',axis=1),output
x_train,y_train=separate(train)
x_test,y_test=separate(test)

In [50]:
y = pd.DataFrame(data['Exited'])
x = data.drop(['Exited'], axis=1)

In [51]:
class MultiNeuralNetwork:
    def __init__(self):
        self.learning_rate=0.1
        self.epoch=1000
        
    def train(self,x_train_data,y_train_data):
        layers=[x_train_data.shape[1],3,2,1]
        print("layers",layers)
        A=[0]*len(layers)
        A[0]=x_train_data.T
#             print(A[0])
        z=[0]*len(layers)
        dg=[0]*len(layers)
        da=[0]*len(layers)
        dz=[0]*len(layers)
        dw=[0]*len(layers)
        db=[0]*len(layers)
        da1=[0]*len(layers)
        da2=[0]*len(layers)
        weight=[]
        bias=[]

         # Initializing weights and bias for all layers 
        for i in range(len(layers)):
            weight.append((np.random.randn(layers[i],layers[i-1])*0.1))
#             print(weight)
            bias.append(np.zeros((layers[i],1)))
#             print(bias)
        
        for data in range(self.epoch):
        #forward propogation
            for i in range(1,len(layers)):
                #hypothesis function
                z[i]=np.dot(weight[i],A[i-1])+bias[i]
#                 print("weight",weight[i].shape)
#                 print("z",z[i].shape)
                # Activation 
                A[i]=np.divide(1,np.add(1,np.exp(-z[i])))
#                 print("activation",A[i].shape)
                
        #backward propogation
            for j in reversed(range(1,len(layers))):
                #derivative of sigmoid
                dg[j]=np.multiply(A[j],(1-A[j]))
#                 print("derivation of dg",dg[j].shape)
                
#                 da[j] = ((-y_train_data[j]/A[j])+((1-y_train_data[j])/(1-A[j])))
                da1[j]=np.divide(-y_train_data[j],A[j])
#                 print("da1",da1[j].shape)
                da2[j]=np.divide((1-y_train_data[j]),(1-A[j]))
#                 print("da2",da2[j].shape)
                
                da[j]=np.add(da1[j],da2[j])
#                 print("derivation of da",da[j].shape)
                
                dz[j]=np.multiply(dg[j],da[j])
#                 print("derivation of dz",dz[j].shape)
                
                dw[j]=np.dot(dz[j],A[j-1].T)/len(x_train_data)
#                 print("derivation of dw",dw[j].shape)
                
                weight[j]=weight[j]-np.dot(self.learning_rate,dw[j])
#                 print("weight",weight[j].shape)
                
                bias[j]=bias[j]-np.dot(self.learning_rate,db[j])
#                 print("bias",bias[j].shape)
                
        return weight,bias
        
        
    def test(self,x_test_data,weight,bias):
        layers=[x_test_data.shape[1],3,2,1]
        z=[0]*len(layers)
        A=[0]*len(layers)
        A[0]=x_test_data.T
        for i in range(1,len(layers)):
            #hypothesis function
            z[i]=np.dot(weight[i],A[i-1])+bias[i]
#             print("z",z[i].shape)
 
            # Activation 
            A[i]=np.divide(1,np.add(1,np.exp(-z[i])))
#             print("activation",A[i].shape)   
        return A[-1]
    
    def accuracy(self,y_test_data,y_predict):
        accuracy=100-(np.mean(np.abs(y_predict-y_test_data))*100)
        return accuracy
        
def main():
    obj=MultiNeuralNetwork()
    
    x_train_data=np.array(x_train)
    print("x train data",x_train_data.shape)
    y_train_data=np.array(y_train)
    y_train_data=y_train_data.reshape(len(y_train_data),1)
    print("y_train_data",y_train_data.shape)
    
    
    x_test_data=np.array(x_test)
    print("x test data",x_test_data.shape)
    y_test_data=np.array(y_test)
    y_test_data=y_test_data.reshape(len(y_test_data),1)
    print("y_test_data",y_test_data.shape)
    

    weight,bias=obj.train(x_train_data,y_train_data)
#     print("weight",weight)
#     print("bias",bias)
    
    y_predict=obj.test(x_test_data,weight,bias)
    print("predication",y_predict)
    
    
    final_accuracy=obj.accuracy(y_test_data,y_predict)
    print("Accurancy-->",final_accuracy)
                      
if __name__=='__main__':
    main()

x train data (7000, 2945)
y_train_data (7000, 1)
x test data (3000, 2945)
y_test_data (3000, 1)
layers [2945, 3, 2, 1]
predication [[0.02104314 0.0196003  0.02050721 ... 0.02045243 0.02072236 0.02069499]]
Accurancy--> 78.95863477591419
